In [1]:
import re

def xplit(*delimiters):
    return lambda value: re.split('|'.join([re.escape(delimiter) for delimiter in delimiters]), value)

In [2]:
xplit('. ', '? ', '! ', '\n', '.\n')("This is a sentence. Here is another sentence.\nHello, world!")

['This is a sentence', 'Here is another sentence', 'Hello, world!']

In [3]:
from konlpy.tag import Kkma

In [5]:
kkma = Kkma()
kkma.sentences("이세돌은 알파고를 이겼다. 이세돌은 강하다. 알파고도 짱쎔.")

['이세 돌은 알 파고를 이겼다.', '이세 돌은 강하다.', '알 파고도 짱 쎔.']

In [6]:
kkma.nouns("눈보라에 차갑게 얼린 머리카락 먹고싶다.")

['눈보라', '머리카락']

kkma는 중복을 제거하기 때문에 twitter 사용

In [7]:
from konlpy.tag import Twitter

twitter = Twitter()
twitter.nouns("미쿠 미쿠 하게 해줄게")
a# ['미쿠', '미쿠']

['미쿠', '미쿠']

### 두 문장(노드)의 연관성을 어떻게 측정할 것인가? -> 유사도? ->두 문장이 가진 단어

In [8]:
from collections import Counter

bow1 = Counter(twitter.nouns("미쿠 미쿠 하게 해줄게"))
# Counter({'미쿠': 2})
bow2 = Counter(twitter.nouns("미쿠 머리카락 맛있겠다"))
# Counter({'미쿠': 1, '머리카락': 1})

In [10]:
bow1&bow2

Counter({'미쿠': 1})

In [11]:
bow1|bow2

Counter({'머리카락': 1, '미쿠': 2})

In [13]:
j_index = sum((bow1 & bow2).values()) / sum((bow1 | bow2).values())
j_index

0.3333333333333333

### 실행


In [15]:
class Sentence:
    
    def __init__(self, text, index=0):
        self.index = index
        self.text = text
        self.nouns = twitter.nouns(self.text)
        self.bow = Counter(self.nouns)
        
    def similarity(s1, s2):
        p = sum((s1.bow&s2.bow).values())
        q = sum((s1.bow|s2.bow).values())
        return p/q if q else 0
        
    def __eq__(self, another):
        return hasattr(another, 'index') and self.index == another.index
    
    def __hash__(self):
        return self.index


In [16]:
def get_sentences(text):
    candidates = xplit('.', '? ', '! ', '\n', '.\n')(text.strip())
    sentences=[]
    index=0
    for candidate in candidates:
        candidate = candidate.strip()
        if len(candidate):
            sentences.append(Sentence(candidate, index))
            index += 1
    return sentences

In [17]:
test = "트위터, \"정보당국에 데이터 분석자료 팔지 않겠다\". 트위터가 수많은 트윗을 분석해 정보를 판매하는 서비스를 미국 정보당국에는 제공하지 않기로 했다. 월스트리트저널은 미국 정보당국 관계자 등을 인용해 데이터마이너(Dataminer)가 정보당국에 대한 서비스는 중단하기로 했다고 9일(현지시간) 보도했다. 트위터가 5% 지분을 가진 데이터마이너는 소셜미디어상 자료를 분석해 고객이 의사결정을 하도록 정보를 제공하는 기업이다. 트위터에 올라오는 트윗에 실시간으로 접근해 분석한 자료를 고객에게 팔 수 있는 독점권을 갖고 있다. 정보당국은 이 회사로부터 구매한 자료로 테러나 정치적 불안정 등과 관련된 정보를 획득했다. 이 회사가 정보당국에 서비스를 판매하지 않기로 한 것은 트위터의 결정인 것으로 알려졌다. 데이터마이너 경영진은 최근 “트위터가 정보당국에 서비스하는 것을 원치 않는다”고 밝혔다고 이 신문은 전했다. 트위터도 성명을 내고 “정보당국 감시용으로 데이터를 팔지 않는 것은 트위터의 오래된 정책”이라며 “트위터 자료는 대체로 공개적이고 미국 정부도 다른 사용자처럼 공개된 어카운트를 살펴볼 수 있다”고 해명했다. 그러나 이는 이 회사가 2년 동안 정보당국에 서비스를 제공해 온 데 대해서는 타당한 설명이 되지 않는다. 트위터의 이번 결정은 미국의 정보기술(IT)기업과 정보당국 간 갈등의 연장 선상에서 이뤄진 것으로 여겨지고 있다. IT기업은 이용자 프라이버시에 무게 중심을 두는 데 비해 정보당국은 공공안전을 우선시해 차이가 있었다. 특히 애플은 캘리포니아 주 샌버너디노 총격범의 아이폰에 저장된 정보를 보겠다며 데이터 잠금장치 해제를 요구하는 미 연방수사국(FBI)과 소송까지 진행했다. 정보당국 고위 관계자도 “트위터가 정보당국과 너무 가까워 보이는 것을 우려하는 것 같다”고 말했다. 데이터마이너는 금융기관이나, 언론사 등 정보당국을 제외한 고객에 대한 서비스는 계속할 계획이다."

test

'트위터, "정보당국에 데이터 분석자료 팔지 않겠다". 트위터가 수많은 트윗을 분석해 정보를 판매하는 서비스를 미국 정보당국에는 제공하지 않기로 했다. 월스트리트저널은 미국 정보당국 관계자 등을 인용해 데이터마이너(Dataminer)가 정보당국에 대한 서비스는 중단하기로 했다고 9일(현지시간) 보도했다. 트위터가 5% 지분을 가진 데이터마이너는 소셜미디어상 자료를 분석해 고객이 의사결정을 하도록 정보를 제공하는 기업이다. 트위터에 올라오는 트윗에 실시간으로 접근해 분석한 자료를 고객에게 팔 수 있는 독점권을 갖고 있다. 정보당국은 이 회사로부터 구매한 자료로 테러나 정치적 불안정 등과 관련된 정보를 획득했다. 이 회사가 정보당국에 서비스를 판매하지 않기로 한 것은 트위터의 결정인 것으로 알려졌다. 데이터마이너 경영진은 최근 “트위터가 정보당국에 서비스하는 것을 원치 않는다”고 밝혔다고 이 신문은 전했다. 트위터도 성명을 내고 “정보당국 감시용으로 데이터를 팔지 않는 것은 트위터의 오래된 정책”이라며 “트위터 자료는 대체로 공개적이고 미국 정부도 다른 사용자처럼 공개된 어카운트를 살펴볼 수 있다”고 해명했다. 그러나 이는 이 회사가 2년 동안 정보당국에 서비스를 제공해 온 데 대해서는 타당한 설명이 되지 않는다. 트위터의 이번 결정은 미국의 정보기술(IT)기업과 정보당국 간 갈등의 연장 선상에서 이뤄진 것으로 여겨지고 있다. IT기업은 이용자 프라이버시에 무게 중심을 두는 데 비해 정보당국은 공공안전을 우선시해 차이가 있었다. 특히 애플은 캘리포니아 주 샌버너디노 총격범의 아이폰에 저장된 정보를 보겠다며 데이터 잠금장치 해제를 요구하는 미 연방수사국(FBI)과 소송까지 진행했다. 정보당국 고위 관계자도 “트위터가 정보당국과 너무 가까워 보이는 것을 우려하는 것 같다”고 말했다. 데이터마이너는 금융기관이나, 언론사 등 정보당국을 제외한 고객에 대한 서비스는 계속할 계획이다.'

In [20]:
get_sentences(test)

In [28]:
def build_graph(sentences):
    graph = networkx.Graph()
    graph.add_nodes_from(sentences)
    pairs = list(itertools.combinations(sentences,2))
    for eins, zwei in pairs:
        graph.add_edge(eins, zwei, weight=Sentence.similarity(eins,zwei))
    return graph

In [22]:
sentences = get_sentences(test)

In [26]:
sdf

NameError: name 'sdf' is not defined

In [ ]:
import itertools
import networkx

In [29]:
graph = build_graph(sentences)

In [30]:
graph

In [31]:
pagerank = networkx.pagerank(graph, weight='weight')

In [32]:
pagerank

{<__main__.Sentence at 0x122983470>: 0.08776661504503318,
 <__main__.Sentence at 0x122983518>: 0.08983809636262718,
 <__main__.Sentence at 0x12297b358>: 0.0711975974038375,
 <__main__.Sentence at 0x12297b0b8>: 0.07016183178278464,
 <__main__.Sentence at 0x12297b5c0>: 0.03648601385442118,
 <__main__.Sentence at 0x12297b438>: 0.05936487367089252,
 <__main__.Sentence at 0x12297b7b8>: 0.09114645342276333,
 <__main__.Sentence at 0x12297b6a0>: 0.08510321024336112,
 <__main__.Sentence at 0x12297b908>: 0.05800323832138602,
 <__main__.Sentence at 0x12297ba90>: 0.06198560441602005,
 <__main__.Sentence at 0x12297bac8>: 0.06763037330500797,
 <__main__.Sentence at 0x12297ba20>: 0.046424654351801715,
 <__main__.Sentence at 0x12297b8d0>: 0.03015071750974774,
 <__main__.Sentence at 0x12297b898>: 0.0728288945361149,
 <__main__.Sentence at 0x12297b5f8>: 0.0719118257742012}

In [33]:
reordered = sorted(pagerank, key=pagerank.get, reverse=True)

In [34]:
reordered

In [36]:
reordered[0].text

'이 회사가 정보당국에 서비스를 판매하지 않기로 한 것은 트위터의 결정인 것으로 알려졌다'

In [37]:
for t in reordered:
    print(t.text)

이 회사가 정보당국에 서비스를 판매하지 않기로 한 것은 트위터의 결정인 것으로 알려졌다
트위터가 수많은 트윗을 분석해 정보를 판매하는 서비스를 미국 정보당국에는 제공하지 않기로 했다
트위터, "정보당국에 데이터 분석자료 팔지 않겠다"
데이터마이너 경영진은 최근 “트위터가 정보당국에 서비스하는 것을 원치 않는다”고 밝혔다고 이 신문은 전했다
정보당국 고위 관계자도 “트위터가 정보당국과 너무 가까워 보이는 것을 우려하는 것 같다”고 말했다
데이터마이너는 금융기관이나, 언론사 등 정보당국을 제외한 고객에 대한 서비스는 계속할 계획이다
월스트리트저널은 미국 정보당국 관계자 등을 인용해 데이터마이너(Dataminer)가 정보당국에 대한 서비스는 중단하기로 했다고 9일(현지시간) 보도했다
트위터가 5% 지분을 가진 데이터마이너는 소셜미디어상 자료를 분석해 고객이 의사결정을 하도록 정보를 제공하는 기업이다
트위터의 이번 결정은 미국의 정보기술(IT)기업과 정보당국 간 갈등의 연장 선상에서 이뤄진 것으로 여겨지고 있다
그러나 이는 이 회사가 2년 동안 정보당국에 서비스를 제공해 온 데 대해서는 타당한 설명이 되지 않는다
정보당국은 이 회사로부터 구매한 자료로 테러나 정치적 불안정 등과 관련된 정보를 획득했다
트위터도 성명을 내고 “정보당국 감시용으로 데이터를 팔지 않는 것은 트위터의 오래된 정책”이라며 “트위터 자료는 대체로 공개적이고 미국 정부도 다른 사용자처럼 공개된 어카운트를 살펴볼 수 있다”고 해명했다
IT기업은 이용자 프라이버시에 무게 중심을 두는 데 비해 정보당국은 공공안전을 우선시해 차이가 있었다
트위터에 올라오는 트윗에 실시간으로 접근해 분석한 자료를 고객에게 팔 수 있는 독점권을 갖고 있다
특히 애플은 캘리포니아 주 샌버너디노 총격범의 아이폰에 저장된 정보를 보겠다며 데이터 잠금장치 해제를 요구하는 미 연방수사국(FBI)과 소송까지 진행했다
